<a href="https://colab.research.google.com/github/marianatiele/etl_aws_combustiveis/blob/main/Notebook_Silver_To_Gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Silver to Gold

## **Baixando o Pyspark**

In [ ]:
!pip install boto3

In [ ]:
!pip install findspark

In [ ]:
!pip install pyngrok

In [ ]:
# instalar o Spark
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  # Instalação do Java

# Baixar a versão do Spark desejada (Spark 3.5.4 com Hadoop 3)
!wget -q https://archive.apache.org/dist/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz

# Extrair os arquivos do Spark
!tar xf spark-3.5.4-bin-hadoop3.tgz


## **Importando as Bibliotecas**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyngrok import ngrok
import findspark
import os
import boto3
from io import BytesIO

In [ ]:
# configurar as variáveis de ambiente

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

aws_access_key_id = "SEU ACCESS KEY"
aws_secret_access_key = "SEU SECRET ACCESS KEY"
region_name = "SUA REGIAO"

In [ ]:
findspark.init()

In [ ]:
#  Iniciando a sessão Spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [ ]:
boto3.setup_default_session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
)
s3 = boto3.client('s3')

In [ ]:
s3.download_file('datalake-combustiveis-br', 'Bronze/precos-gasolina-etanol-01.csv', 'precos-gasolina-etanol-01.csv')
s3.download_file('datalake-combustiveis-br', 'Bronze/precos-gasolina-etanol-02.csv', 'precos-gasolina-etanol-02.csv')
s3.download_file('datalake-combustiveis-br', 'Bronze/precos-gasolina-etanol-03.csv', 'precos-gasolina-etanol-03.csv')

In [ ]:
dados = spark.read.csv('precos-gasolina-etanol-01.csv', header=True, inferSchema=True, sep=';')
dados_2 = spark.read.csv('precos-gasolina-etanol-02.csv', header=True, inferSchema=True, sep=';')
dados_3 = spark.read.csv('precos-gasolina-etanol-03.csv', header=True, inferSchema=True, sep=';')

In [ ]:
dados.show()

+------+------+---------+--------------------+-------------------+--------------------+------+-----------+--------------------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                Nome|Numero|Complemento|              Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Valor_ Compra|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+-----------+--------------------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|       NULL|              CENTRO|62010-000|          GASOLINA| 01/01/2025|       6,29|         NULL|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|       NULL|              CENT

In [ ]:
dados_2.show()

+------+------+---------------+--------------------+-------------------+--------------------+------+-----------+---------------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|Regiao|Estado|      Municipio|             Revenda|               CNPJ|                Nome|Numero|Complemento|         Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Valor_ Compra|Unidade_Medida|Bandeira|
+------+------+---------------+--------------------+-------------------+--------------------+------+-----------+---------------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|       NULL|         CENTRO|69980-000|          GASOLINA| 03/02/2025|       7,27|         NULL|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|       NULL|        

In [ ]:
dados_3.show()

+------+------+---------------+--------------------+-------------------+--------------------+------+-----------+---------------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|Regiao|Estado|      Municipio|             Revenda|               CNPJ|                Nome|Numero|Complemento|         Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Valor_ Compra|Unidade_Medida|Bandeira|
+------+------+---------------+--------------------+-------------------+--------------------+------+-----------+---------------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|    SE|    SP|       SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|       NULL| JARDIM ZULMIRA|18061-000|          GASOLINA| 03/03/2025|       5,65|         NULL|    R$ / litro|  BRANCA|
|    SE|    SP|       SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|       NULL| JARDIM 

In [ ]:
len(dados.columns), len(dados_2.columns), len(dados_3.columns)

(16, 16, 16)

# Tratamento dos dados

In [ ]:
dados = dados.withColumnRenamed('Valor_ Compra', 'Valor_Compra')

In [ ]:
colunas = dados.columns
colunas[13]

'Valor_Compra'

In [ ]:
dados_2 = (dados_2.withColumnRenamed('Regiao - Sigla', colunas[0])
    .withColumnRenamed('Estado - Sigla', colunas[1])
    .withColumnRenamed('CNPJ da Revenda', colunas[4])
    .withColumnRenamed('Nome da Rua', colunas[5])
    .withColumnRenamed('Numero Rua', colunas[6])
    .withColumnRenamed('Data da Coleta', colunas[11])
    .withColumnRenamed('Valor de Venda', colunas[12])
    .withColumnRenamed('Valor de Compra', colunas[13])
    .withColumnRenamed('Unidade de Medida', colunas[14])
)



In [ ]:
dados_3 = (dados_3.withColumnRenamed('Regiao - Sigla', colunas[0])
    .withColumnRenamed('Estado - Sigla', colunas[1])
    .withColumnRenamed('CNPJ da Revenda', colunas[4])
    .withColumnRenamed('Nome da Rua', colunas[5])
    .withColumnRenamed('Numero Rua', colunas[6])
    .withColumnRenamed('Data da Coleta', colunas[11])
    .withColumnRenamed('Valor de Venda', colunas[12])
    .withColumnRenamed('Valor de Compra', colunas[13])
    .withColumnRenamed('Unidade de Medida', colunas[14])
)

In [ ]:
# Decimal tipo americano
dados = dados.withColumn("Valor_Venda", regexp_replace(col("Valor_Venda"), ",", ".").cast("float"))
dados_2 = dados_2.withColumn("Valor_Venda", regexp_replace(col("Valor_Venda"), ",", ".").cast("float"))
dados_3 = dados_3.withColumn("Valor_Venda", regexp_replace(col("Valor_Venda"), ",", ".").cast("float"))

dados = dados.withColumn("Numero", regexp_replace(col("Numero"), " ", "S/N"))
dados_2 = dados_2.withColumn("Numero", regexp_replace(col("Numero"), " ", "S/N"))
dados_3 = dados_3.withColumn("Numero", regexp_replace(col("Numero"), " ", "S/N"))


In [ ]:
# Apagar coluna inteira null
dados = dados.drop('Valor_Compra')
dados_2 = dados_2.drop('Valor_Compra')
dados_3 = dados_3.drop('Valor_ Compra')

dados = dados.drop('Complemento')
dados_2 = dados_2.drop('Complemento')
dados_3 = dados_3.drop('Complemento')

In [ ]:
dados.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                Nome|Numero|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|          GASOLINA| 01/01/2025|       6.29|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|GASOLINA ADITIVADA| 01/01/2025|       6.49|    R$ / litro|  RAIZEN|
+------+------+---------+--------------------+-------------------+--------------------+------+------+----

In [ ]:
dados_2.show(2)

+------+------+---------------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|Regiao|Estado|      Municipio|             Revenda|               CNPJ|                Nome|Numero|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Valor_ Compra|Unidade_Medida|Bandeira|
+------+------+---------------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+-------------+--------------+--------+
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|CENTRO|69980-000|          GASOLINA| 03/02/2025|       7.27|         NULL|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|CENTRO|69980-000|GASOLINA ADITIVADA| 03/02/2025|       7.73|         NULL|    R$ / litro|   VIBRA|
+----

In [ ]:
dados.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                Nome|Numero|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|          GASOLINA| 01/01/2025|       6.29|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|GASOLINA ADITIVADA| 01/01/2025|       6.49|    R$ / litro|  RAIZEN|
+------+------+---------+--------------------+-------------------+--------------------+------+------+----

In [ ]:
dados.select(count(col('Numero'))).distinct().show()

+-------------+
|count(Numero)|
+-------------+
|        46491|
+-------------+



In [ ]:
dados.select('Numero', 'Bairro').where(dados.Numero == 'S/N').show()

+------+--------------------+
|Numero|              Bairro|
+------+--------------------+
|   S/N|          INDUSTRIAL|
|   S/N|          INDUSTRIAL|
|   S/N|            IBITIOCA|
|   S/N|            IBITIOCA|
|   S/N|CHAC. REUN. IGARAPES|
|   S/N|CHAC. REUN. IGARAPES|
|   S/N|CHAC. REUN. IGARAPES|
|   S/N|          ZONA RURAL|
|   S/N|          ZONA RURAL|
|   S/N|            IBITIOCA|
|   S/N|            IBITIOCA|
|   S/N|            IBITIOCA|
|   S/N|   TORRAO DE OURO II|
|   S/N|   TORRAO DE OURO II|
|   S/N|   TORRAO DE OURO II|
|   S/N| DISTRITO INDUSTRIAL|
|   S/N| DISTRITO INDUSTRIAL|
|   S/N| DISTRITO INDUSTRIAL|
|   S/N|                SEDE|
|   S/N|                SEDE|
+------+--------------------+
only showing top 20 rows



In [ ]:
dados.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                Nome|Numero|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|          GASOLINA| 01/01/2025|       6.29|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|GASOLINA ADITIVADA| 01/01/2025|       6.49|    R$ / litro|  RAIZEN|
+------+------+---------+--------------------+-------------------+--------------------+------+------+----

In [ ]:
dados_3.printSchema()

root
 |-- Regiao: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Numero: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data_Coleta: string (nullable = true)
 |-- Valor_Venda: float (nullable = true)
 |-- Unidade_Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [ ]:
# dados.select("Data_Coleta").distinct().show(truncate=False)

In [ ]:
dados  =  (
    dados.withColumn('Data_Coleta', to_date(col('Data_Coleta'), "dd/MM/yyyy"))
    .withColumn('Valor_Venda', col('Valor_Venda').cast('decimal(5,2)'))
    )


In [ ]:
dados.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                Nome|Numero|Bairro|      Cep|           Produto|Data_Coleta|Valor_Venda|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|          GASOLINA| 2025-01-01|       6.29|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|GASOLINA ADITIVADA| 2025-01-01|       6.49|    R$ / litro|  RAIZEN|
+------+------+---------+--------------------+-------------------+--------------------+------+------+----

In [ ]:
dados.printSchema()

root
 |-- Regiao: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Numero: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data_Coleta: date (nullable = true)
 |-- Valor_Venda: decimal(5,2) (nullable = true)
 |-- Unidade_Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [ ]:
dados_2  =  (
    dados_2.withColumn('Data_Coleta', to_date(col('Data_Coleta'), "dd/MM/yyyy"))
    .withColumn('Valor_Venda', col('Valor_Venda').cast('decimal(5,2)'))
    )

In [ ]:
dados_3  =  (
   dados_3.withColumn('Data_Coleta', to_date(col('Data_Coleta'), "dd/MM/yyyy"))
    .withColumn('Valor_Venda', col('Valor_Venda').cast('decimal(5,2)'))
    )

In [ ]:
dados_2.printSchema()

root
 |-- Regiao: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Numero: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data_Coleta: date (nullable = true)
 |-- Valor_Venda: decimal(5,2) (nullable = true)
 |-- Valor_ Compra: string (nullable = true)
 |-- Unidade_Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [ ]:
dados_3.printSchema()

root
 |-- Regiao: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Numero: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data_Coleta: date (nullable = true)
 |-- Valor_Venda: decimal(5,2) (nullable = true)
 |-- Unidade_Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [ ]:
dados_3.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|Regiao|Estado|Municipio|             Revenda|               CNPJ|                Nome|Numero|        Bairro|      Cep| Produto|Data_Coleta|Valor_Venda|Unidade_Medida|Bandeira|
+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|GASOLINA| 2025-03-03|       5.65|    R$ / litro|  BRANCA|
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|  ETANOL| 2025-03-03|       3.78|    R$ / litro|  BRANCA|
+------+------+---------+--------------------+-------------------+--------------------+------+--------------+------

In [ ]:
dados.columns

['Regiao',
 'Estado',
 'Municipio',
 'Revenda',
 'CNPJ',
 'Nome',
 'Numero',
 'Bairro',
 'Cep',
 'Produto',
 'Data_Coleta',
 'Valor_Venda',
 'Unidade_Medida',
 'Bandeira']

In [ ]:
dados = dados.select(
'Regiao'.upper(),
 'Estado'.upper(),
 'Municipio'.upper(),
 'Revenda'.upper(),
 'CNPJ'.upper(),
 'Nome'.upper(),
 'Numero'.upper(),
 'Bairro'.upper(),
 'Cep'.upper(),
 'Produto'.upper(),
 'Data_Coleta'.upper(),
 'Valor_Venda'.upper(),
 'Unidade_Medida'.upper(),
 'Bandeira'.upper()
)


In [ ]:
dados_2 = dados_2.select(
'Regiao'.upper(),
 'Estado'.upper(),
 'Municipio'.upper(),
 'Revenda'.upper(),
 'CNPJ'.upper(),
 'Nome'.upper(),
 'Numero'.upper(),
 'Bairro'.upper(),
 'Cep'.upper(),
 'Produto'.upper(),
 'Data_Coleta'.upper(),
 'Valor_Venda'.upper(),
 'Unidade_Medida'.upper(),
 'Bandeira'.upper()
)


In [ ]:
dados_2.printSchema()

root
 |-- REGIAO: string (nullable = true)
 |-- ESTADO: string (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- REVENDA: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- NOME: string (nullable = true)
 |-- NUMERO: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- PRODUTO: string (nullable = true)
 |-- DATA_COLETA: date (nullable = true)
 |-- VALOR_VENDA: decimal(5,2) (nullable = true)
 |-- UNIDADE_MEDIDA: string (nullable = true)
 |-- BANDEIRA: string (nullable = true)



In [ ]:
dados_3 = dados_3.select(
'Regiao'.upper(),
 'Estado'.upper(),
 'Municipio'.upper(),
 'Revenda'.upper(),
 'CNPJ'.upper(),
 'Nome'.upper(),
 'Numero'.upper(),
 'Bairro'.upper(),
 'Cep'.upper(),
 'Produto'.upper(),
 'Data_Coleta'.upper(),
 'Valor_Venda'.upper(),
 'Unidade_Medida'.upper(),
 'Bandeira'.upper()
)


In [ ]:
display(dados.show(5))

+------+------+---------+--------------------+-------------------+--------------------+------+----------------+---------+------------------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|             REVENDA|               CNPJ|                NOME|NUMERO|          BAIRRO|      CEP|           PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+--------------------+-------------------+--------------------+------+----------------+---------+------------------+-----------+-----------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|          CENTRO|62010-000|          GASOLINA| 2025-01-01|       6.29|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|          CENTRO|62010-000|GASOLINA ADITIVADA| 2025-01-01|       6.49|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.9

None

In [ ]:
display(dados_2.show(5))

+------+------+---------------+--------------------+-------------------+--------------------+------+----------+---------+------------------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|      MUNICIPIO|             REVENDA|               CNPJ|                NOME|NUMERO|    BAIRRO|      CEP|           PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------------+--------------------+-------------------+--------------------+------+----------+---------+------------------+-----------+-----------+--------------+--------+
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|    CENTRO|69980-000|          GASOLINA| 2025-02-03|       7.27|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|    CENTRO|69980-000|GASOLINA ADITIVADA| 2025-02-03|       7.73|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...| 01

None

In [ ]:
display(dados_3.show(5))

+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|             REVENDA|               CNPJ|                NOME|NUMERO|        BAIRRO|      CEP| PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|GASOLINA| 2025-03-03|       5.65|    R$ / litro|  BRANCA|
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|  ETANOL| 2025-03-03|       3.78|    R$ / litro|  BRANCA|
|    NE|    AL|ARAPIRACA|AUTO POSTO M M GA...| 08.738.994/0001-50|      RODOVIA AL-220|  5848|   BOM SUCESSO|57309-

None

In [ ]:
len(dados.columns), dados.count()

(14, 46509)

In [ ]:
len(dados_2.columns), dados_2.count()

(14, 44290)

In [ ]:
len(dados_3.columns), dados_3.count()

(14, 51685)

In [ ]:
display(dados.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados.columns]).show())

+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|NOME|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|   0|    18|    84|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+



None

In [ ]:
dados = dados.na.drop()

In [ ]:
display(dados.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados.columns]).show())

+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|NOME|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|   0|     0|     0|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+



None

In [ ]:
display(dados_2.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados_2.columns]).show())

+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|NOME|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|   0|     3|    91|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+



None

In [ ]:
display(dados_3.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados_3.columns]).show())

+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|NOME|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|   0|     9|   114|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+



None

In [ ]:
dados_2 = dados_2.na.drop()
dados_3 = dados_3.na.drop()

In [ ]:
dados_2.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados_2.columns]).show()

+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|NOME|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|   0|     0|     0|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+



In [ ]:
dados_3.select([sum(col(c).isNull().cast("int")).alias(c) for c in dados_3.columns]).show()

+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|REVENDA|CNPJ|NOME|NUMERO|BAIRRO|CEP|PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+
|     0|     0|        0|      0|   0|   0|     0|     0|  0|      0|          0|          0|             0|       0|
+------+------+---------+-------+----+----+------+------+---+-------+-----------+-----------+--------------+--------+



In [ ]:
display(dados_2.show(5))

+------+------+---------------+--------------------+-------------------+--------------------+------+----------+---------+------------------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|      MUNICIPIO|             REVENDA|               CNPJ|                NOME|NUMERO|    BAIRRO|      CEP|           PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------------+--------------------+-------------------+--------------------+------+----------+---------+------------------+-----------+-----------+--------------+--------+
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|    CENTRO|69980-000|          GASOLINA| 2025-02-03|       7.27|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|CARDINAL E OLIVEI...| 03.608.766/0005-75|AVENIDA BOULEVARD...|  1031|    CENTRO|69980-000|GASOLINA ADITIVADA| 2025-02-03|       7.73|    R$ / litro|   VIBRA|
|     N|    AC|CRUZEIRO DO SUL|AMAZONIA COMERCIO...| 01

None

In [ ]:
display(dados_3.show(5))

+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|             REVENDA|               CNPJ|                NOME|NUMERO|        BAIRRO|      CEP| PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|GASOLINA| 2025-03-03|       5.65|    R$ / litro|  BRANCA|
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|  ETANOL| 2025-03-03|       3.78|    R$ / litro|  BRANCA|
|    NE|    AL|ARAPIRACA|AUTO POSTO M M GA...| 08.738.994/0001-50|      RODOVIA AL-220|  5848|   BOM SUCESSO|57309-

None

In [ ]:
# dados = dados.coalesce(1)

In [ ]:
display(dados_3.show(2))

+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|             REVENDA|               CNPJ|                NOME|NUMERO|        BAIRRO|      CEP| PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+--------------------+-------------------+--------------------+------+--------------+---------+--------+-----------+-----------+--------------+--------+
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|GASOLINA| 2025-03-03|       5.65|    R$ / litro|  BRANCA|
|    SE|    SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|   306|JARDIM ZULMIRA|18061-000|  ETANOL| 2025-03-03|       3.78|    R$ / litro|  BRANCA|
+------+------+---------+--------------------+-------------------+--------------------+------+--------------+------

None

In [ ]:
dados.show(2)

+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|REGIAO|ESTADO|MUNICIPIO|             REVENDA|               CNPJ|                NOME|NUMERO|BAIRRO|      CEP|           PRODUTO|DATA_COLETA|VALOR_VENDA|UNIDADE_MEDIDA|BANDEIRA|
+------+------+---------+--------------------+-------------------+--------------------+------+------+---------+------------------+-----------+-----------+--------------+--------+
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|          GASOLINA| 2025-01-01|       6.29|    R$ / litro|  RAIZEN|
|    NE|    CE|   SOBRAL|ECONOGÁS DO BRASI...| 08.775.979/0002-62|RUA TABELIÃO IDEL...|   455|CENTRO|62010-000|GASOLINA ADITIVADA| 2025-01-01|       6.49|    R$ / litro|  RAIZEN|
+------+------+---------+--------------------+-------------------+--------------------+------+------+----

## **Salvando dados da Camada Silver na Amazon S3**

In [ ]:
# criando diretório local
!mkdir dados_csv

In [ ]:
dados.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true').option('inferSchema','true').save("dados_csv/preco-gasolina-etanol-01")
dados_2.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true').option('inferSchema','true').save("dados_csv/preco-gasolina-etanol-02")
dados_3.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true').option('inferSchema','true').save("dados_csv/preco-gasolina-etanol-03")

In [ ]:
s3.upload_file(
    '/content/dados_csv/preco-gasolina-etanol-01/part-00000-3a9eeae4-658f-4331-8fc2-7492fb230e89-c000.csv',
    'datalake-combustiveis-br',
    'Silver/preco-gasolina-etanol-01.csv')



In [ ]:
s3.upload_file(
    '/content/dados_csv/preco-gasolina-etanol-02/part-00000-13bf33ae-3abb-4631-9afb-3241da429600-c000.csv',
    'datalake-combustiveis-br',
    'Silver/preco-gasolina-etanol-02.csv')



In [ ]:
s3.upload_file('/content/dados_csv/preco-gasolina-etanol-03/part-00000-590813d4-262d-47f5-ad24-9b440b8faa27-c000.csv',
               'datalake-combustiveis-br',
               'Silver/preco-gasolina-etanol-03.csv')